In [ ]:
from scipy.stats import poisson
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# 1

Сортируем по возрастанию, минимальное и максимальное значение соответственно а и b

# 2 

In [ ]:
n = 17
rv = poisson(n)

In [ ]:
df = pd.DataFrame({}, columns=['mean error', 'var error'])
for i in range(15):
    sample = rv.rvs(size=100)
    df = df.append({'mean error': abs(sample.mean() - n),
                    'var error':abs(sample.std()**2 - n)},
                   ignore_index=True)

In [ ]:
df

In [ ]:
for i, j in df.T.iterrows():
    print(i, '=', j.mean())

!wget https://raw.githubusercontent.com/s-a-nersisyan/HSE_bioinformatics_2021/master/seminar11/breast_cancer_1000_genes.tsv

!wget https://raw.githubusercontent.com/s-a-nersisyan/HSE_bioinformatics_2021/master/seminar11/breast_cancer_key_genes.tsv

In [ ]:
df1000 = pd.read_csv('breast_cancer_1000_genes.tsv', sep='\t', index_col=0)
df_key = pd.read_csv('breast_cancer_key_genes.tsv', sep='\t', index_col=0)

# 3 

$\theta$ = mean(Xn)/n (по определению $\theta$ Bin(m, $\theta$) $\theta$ = E(X)/n)

m = max(Xn) (Самый простой вариант)


# 4 

In [ ]:
df1000.head()

In [ ]:
print(plt.style.available)

In [ ]:
df = pd.DataFrame()
df['Difference between maximum and minimum'] = df1000.max(axis=1) - df1000.min(axis=1)
df['Variance'] = df1000.var(axis=1)
df['Interquartile range'] = df1000.quantile(.75, axis=1) - df1000.quantile(.25, axis=1) 

palette = 'mako'

plt.style.use('seaborn-poster')
fig, ax = plt.subplots(3, 2, figsize=(25, 20))
axes = [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1)]
tops = {}

for i, j in zip(df.columns, axes):
    tmp = df.sort_values(i).iloc[:10].index
    for k in tmp:
        tops[k] = tops.get(k, 0) + 1
    sns.boxplot(data=df1000.loc[tmp].T, ax=ax[j], palette=palette)
    
    ax[j].set_title(i)
    ax[j].set_xlabel('Genes')
    ax[j].set_ylabel('Expression')

tops_labels = sorted(tops, key=lambda x: tops[x], reverse=True)
tops_values = [tops[i] for i in tops_labels]

sns.barplot(x=tops_labels, y=tops_values, ax=ax[axes[-3]], alpha=.8, palette=palette)
ax[axes[-3]].set_title('The number of occurrences of genes in different sample metrics')
ax[axes[-3]].set_yticks([i for i in range(1, 4)])
ax[axes[-3]].set_xticks(ax[axes[-3]].get_xticks())
ax[axes[-3]].set_xticklabels(ax[axes[-3]].get_xticklabels(), rotation=45)
ax[axes[-3]].set_xlabel('Genes')
ax[axes[-3]].set_ylabel('Number of occurances')

tmp = df1000.loc[tops_labels[0:5]]
sns.kdeplot(data=tmp.T, ax=ax[axes[-2]], fill=True, common_norm=True, palette=palette)
ax[axes[-2]].set_title('Kernel density estimation plot of most "stable" genes')
ax[axes[-2]].set_xlabel('Expression')
ax[axes[-2]].set_ylabel(' Normalized density')

tmp = tmp.T.melt(var_name='Gene', value_name='Expression')
sns.histplot(data=tmp, x='Expression', hue='Gene',
             binrange=(tmp.Expression.min(), tmp.Expression.max()), bins=20,
             stat='density', palette=palette, ax=ax[axes[-1]])
ax[axes[-1]].set_title('Expression histogram of most "stable" genes')
ax[axes[-1]].set_xlabel(ax[axes[-1]].get_xlabel())
ax[axes[-1]].set_ylabel(ax[axes[-1]].get_ylabel())

plt.tight_layout()
plt.savefig('boxplots_1000', facecolor='white')

# 5 

In [ ]:
df_key.head()

In [ ]:
fig = plt.figure(figsize=(15, 20))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 2, 3)
ax3 = fig.add_subplot(2, 2, 4)

sns.kdeplot(data=df_key, x='PGR', y='ESR1', fill=True,
            thresh=0, levels=3000, cmap="mako", ax=ax1)

df = df_key[['PGR', 'ESR1']]
df.columns = ['PGR expression', 'ESR1 expression']

sns.kdeplot(data=df, palette='mako',
            fill=True, ax=ax2)

sns.histplot(data=df, binrange=(df.min().min(), df.max().max()),
             bins=100, stat='density', palette='mako', ax=ax3)
plt.tight_layout()
fig.savefig('kdeplots', facecolor='white')